In [4]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
def images_preprocessing(path):
    images_paths = []
    images_labels = []
    
    for root, dirs , files in os.walk(path):
        for filename in files:
            if filename.endswith('.jpg'):
                images_paths.append(os.path.join(root, filename))
                images_labels.append(0 if 'dogs' in root.lower() else 1)

    images_paths = np.array(images_paths)
    images_labels = np.array(images_labels)
    
    return images_paths, images_labels

In [3]:
image_train, image_label_train = images_preprocessing("/kaggle/input/cat-and-dog/training_set")
image_test, image_label_test = images_preprocessing("/kaggle/input/cat-and-dog/test_set")

In [4]:
image_train.shape, image_label_train.shape, image_test.shape, image_label_test.shape

((8005,), (8005,), (2023,), (2023,))

In [6]:
def preprocess_images(images_paths, labels_images, target_size=(244, 244)):
    images = []
    labels = []
    
    for i in range(len(images_paths)):
        # Load the image, resize it, and convert to array
        img = load_img(images_paths[i], target_size=target_size)
        img_array = img_to_array(img)
        
        # Normalize pixel values to [0, 1]
        img_array = img_array / 255.0
        
        # Append image and corresponding label
        images.append(img_array)
        labels.append(labels_images[i])  # 0 for Dog, 1 for Cat
    
    return np.array(images), np.array(labels)


In [7]:
image_train_encoded, train_labels_encoded = preprocess_images(image_train, image_label_train)
image_test_encoded, test_labels_encoded = preprocess_images(image_test, image_label_test)

In [8]:
image_test_encoded[-1,243], test_labels_encoded[-1]

(array([[0.79607844, 0.78039217, 0.7372549 ],
        [0.6627451 , 0.6392157 , 0.5921569 ],
        [0.63529414, 0.6117647 , 0.54901963],
        [0.6313726 , 0.59607846, 0.5294118 ],
        [0.6627451 , 0.61960787, 0.54901963],
        [0.6666667 , 0.62352943, 0.5529412 ],
        [0.7176471 , 0.6745098 , 0.6039216 ],
        [0.69411767, 0.6666667 , 0.59607846],
        [0.7254902 , 0.69411767, 0.6431373 ],
        [0.6392157 , 0.62352943, 0.5803922 ],
        [0.64705884, 0.63529414, 0.60784316],
        [0.6509804 , 0.64705884, 0.627451  ],
        [0.69411767, 0.69411767, 0.6627451 ],
        [0.7019608 , 0.7058824 , 0.6745098 ],
        [0.7019608 , 0.7137255 , 0.6784314 ],
        [0.69411767, 0.7176471 , 0.6784314 ],
        [0.7058824 , 0.74509805, 0.7137255 ],
        [0.7176471 , 0.7529412 , 0.73333335],
        [0.5882353 , 0.63529414, 0.63529414],
        [0.45490196, 0.49803922, 0.5058824 ],
        [0.27450982, 0.31764707, 0.33333334],
        [0.26666668, 0.29411766, 0

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [11]:
def CNN_Model(input_shape=(244, 244, 3)):
    MODEL = Sequential([
    # Convolutional Layers
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(3, 3)),

    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.2),  # Regularization
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(3, 3)),

    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.2),  # Regularization
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(3, 3)),

    # Fully Connected Layers
    Flatten(),
    Dense(128, activation='sigmoid'),
    Dense(128, activation='sigmoid'),
    Dropout(0.2),  # Regularization
    Dense(1, activation='sigmoid')  # Binary classification
    ])

    return MODEL

In [16]:
CNN_Model().summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 242, 242, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 240, 240, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 80, 80, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 78, 78, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 78, 78, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 76, 76, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 25, 25, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 23, 23, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 21, 21, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 7, 7, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1568)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         200,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 439,905 (1.68 MB)

 Trainable params: 439,905 (1.68 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
Model = CNN_Model()

Model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary classification
              metrics=['accuracy'])

Model.fit(image_train_encoded,
          train_labels_encoded,
          validation_data=(image_test_encoded, test_labels_encoded),
          batch_size=32,
          epochs=10)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 994s 4s/step - accuracy: 0.5325 - loss: 0.7024 - val_accuracy: 0.5843 - val_loss: 0.6532
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 1011s 4s/step - accuracy: 0.5933 - loss: 0.6504 - val_accuracy: 0.6673 - val_loss: 0.6036
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 1055s 4s/step - accuracy: 0.6659 - loss: 0.6042 - val_accuracy: 0.7168 - val_loss: 0.5473
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 1001s 4s/step - accuracy: 0.7447 - loss: 0.5008 - val_accuracy: 0.7795 - val_loss: 0.4607
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 1005s 4s/step - accuracy: 0.7988 - loss: 0.4339 - val_accuracy: 0.8082 - val_loss: 0.3951
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 1025s 4s/step - accuracy: 0.8317 - loss: 0.3750 - val_accuracy: 0.8329 - val_loss: 0.3742
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 1025s 4s/step - accuracy: 0.8579 - loss: 0.3188 - val_accuracy: 0.8309 - val_loss: 0.3698
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 1058s 4s/step - accuracy: 0.8741 - loss: 0.2915 - v

In [12]:
Model.save('cat_dog_classifier.h5')

In [1]:
from tensorflow.keras.models import load_model

# Load the model
CatDogModel = load_model(r'/kaggle/input/cat-dog-classifier/tensorflow2/default/1/cat_dog_classifier.h5')
print("Model loaded successfully!")

Model loaded successfully!


In [2]:
def predict_image(image_path):
    img = load_img(image_path, target_size=(244, 244))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = CatDogModel.predict(img_array)
    if prediction[0] < 0.5:  # Closer to 0 means Dog
        print("It's a Dog! 🐶")
    else:                    # Closer to 1 means Cat
        print("It's a Cat! 🐱")

In [11]:
predict_image(r'/kaggle/input/ddddddd/2.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
It's a Dog! 🐶


# Loading The Model

In [6]:
cat_dog_model = load_model('/kaggle/input/cat-dog-classifier/tensorflow2/default/1/cat_dog_classifier.h5')

In [7]:
def predict_image(image_path):
    img = load_img(image_path, target_size=(244, 244))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = cat_dog_model.predict(img_array)
    if prediction[0] < 0.5:  # Closer to 0 means Dog
        print("It's a Dog! 🐶")
    else:                    # Closer to 1 means Cat
        print("It's a Cat! 🐱")

In [14]:
predict_image(r'/kaggle/input/dogs/3.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
It's a Dog! 🐶
